### bq 데이터(spark)를 redis 서버에 저장

In [1]:
from pydatafabric.vault_utils import get_secrets

redis_info = get_secrets(mount_point="datafabric", path="redis/datafabric/dev")

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("bq_to_redis_214708") \
    .config("spark.redis.host", redis_info['host']) \
    .config("spark.redis.port", redis_info['port']) \
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
    .config("spark.jars",
            "gs://emart-datafabric-resources/jars/spark-redis_2.12-3.1.0-jar-with-dependencies.jar,hdfs:///jars/spark-bigquery-with-dependencies_default.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/12 16:55:39 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/12 16:55:40 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/12 16:55:40 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/12 16:55:40 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [ ]:
### Spark 세션 생성 후 spark.redis 환경변수 설정은 동작하지 않음
# from pydatafabric.gcp import get_spark

# spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/spark-redis_2.12-3.1.0-jar-with-dependencies.jar")
# spark.conf.set("spark.redis.host", redis_info['host'])
# spark.conf.set("spark.redis.port", redis_info['port'])
# spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [ ]:
df = spark.createDataFrame([{"a": 1}])

In [ ]:
df

In [ ]:
df.write.format("org.apache.spark.sql.redis").option("table", "test").option("key.column", "a").save(mode="append")

In [ ]:
df = spark.read.format("org.apache.spark.sql.redis").option("table", "test").option("key.column", "a").load()
df.show(10)

In [3]:
from pydatafabric.gcp import bq_table_to_df

df = bq_table_to_df("common_dev", "covid_samples", project="emart-datafabric", col_list="*", spark_session=spark)

In [4]:
df.printSchema()

root
 |-- pop_data_2019: long (nullable = true)
 |-- countries_and_territories: string (nullable = true)
 |-- country_territory_code: string (nullable = true)
 |-- geo_id: string (nullable = true)
 |-- deaths: long (nullable = true)
 |-- daily_deaths: long (nullable = true)
 |-- daily_confirmed_cases: long (nullable = true)
 |-- year: long (nullable = true)
 |-- date: date (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- confirmed_cases: long (nullable = true)



In [5]:
df.write.format("org.apache.spark.sql.redis").option("table", "covid_samples").save(mode="append")

In [6]:
df = spark.read.format("org.apache.spark.sql.redis").option("table", "covid_samples").load()
df.show()

+-------------+-------------------------+----------------------+------+------+------------+---------------------+----+----------+-----+---+---------------+
|pop_data_2019|countries_and_territories|country_territory_code|geo_id|deaths|daily_deaths|daily_confirmed_cases|year|      date|month|day|confirmed_cases|
+-------------+-------------------------+----------------------+------+------+------------+---------------------+----+----------+-----+---+---------------+
|       889955|                     Fiji|                   FJI|    FJ|     2|           0|                    0|2020|2020-09-27|    9| 27|             32|
|     15946882|                     Chad|                   TCD|    TD|    74|           0|                    0|2020|2020-07-10|    7| 10|            873|
|       889955|                     Fiji|                   FJI|    FJ|     1|           0|                    0|2020|2020-08-25|    8| 25|             28|
|       889955|                     Fiji|                   FJI|

In [7]:
spark.stop()